#Text rank#


In [0]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx 
from copy import deepcopy


In [0]:
# download and required modules of nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# setting up gdrive 
from google.colab import drive
drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Dataset : tmp#
Description,size


In [0]:
# testing phase 
# will change later to more complex 
# this cell must be well abstracted

articles = {"business":[]}
summaries = {"business":[]}
# file_count = {"business":510,"entertainment":386,"sport":511,"politics":417,"tech":401}
file_count = {"business":510}
for k,v in file_count.items():
  afile = "gdrive/My Drive/BBC_News_Summary/News_Articles/"+k+"/"
  sfile = "/gdrive/My Drive/BBC_News_Summary/Summaries/"+k+"/"
  for i in range(1,1+v):
    afname = afile+("0"*(3-len(str(i))))+str(i)+".txt"
    sfname = afile+("0"*(3-len(str(i))))+str(i)+".txt"
    with open(afname) as f:
      articles[k].append(f.read())
    with open(sfname) as f:
      summaries[k].append(f.read())
  print("done reading ",k)


done reading  business


In [0]:
print(articles["business"][0])

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL

In [0]:
# splitting into sentences
# sentences = deepcopy(articles)
# for k,v in sentences.items():
#   sentences.extend(sent_tokenize(s))
sentences = {"business":[]}
for category,texts in articles.items():
  print("sentence tokenizing ",category)
  for text in texts:
    sentences[category].append(sent_tokenize(text))



sentence tokenizing  business


In [0]:
#print sample sentences
print(sentences["business"][0])

['Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.', 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.', 'TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.', 'Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.', 'Time Warner said on Friday that it now owns 8% of search-engine Google.', 'But its own internet business, AOL, had has mixed fortunes.', 'It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.', "However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.", "It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

#Initialising Word Embeddings(Glove)#

In [0]:
# Uncomment below commands to download zip
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2019-11-24 14:36:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-24 14:36:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-24 14:36:49--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
word_emb = dict()
with open('glove.6B.100d.txt',encoding='utf-8') as f:
  for line in f :
    tokens = line.strip().split()
    word_emb[tokens[0]] = np.asarray(tokens[1:],dtype='float32')
print("Initialised "+str(len(word_emb))+" embeddings")

Initialised 400000 embeddings


#Preprocessing Stage#
* Implement alternate methods for calculating sentence vector.

In [0]:
# text cleaning 

# remove stop_words
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = deepcopy(sentences)
for category,texts in sentences.items():
  for i in range(len(texts)):
    # here texts[i] is list of sentences
    clean_sentences[category][i] = pd.Series(texts[i]).str.replace("[^a-zA-Z]"," ")
    clean_sentences[category][i] = [s.lower() for s in clean_sentences[category][i]]
    clean_sentences[category][i] = [remove_stopwords(r.split()) for r in clean_sentences[category][i]]



In [0]:
print(clean_sentences["business"][0])

['ad sales boost time warner profit quarterly profits us media giant timewarner jumped bn three months december year earlier', 'firm one biggest investors google benefited sales high speed internet connections higher advert sales', 'timewarner said fourth quarter sales rose bn bn', 'profits buoyed one gains offset profit dip warner bros less users aol', 'time warner said friday owns search engine google', 'internet business aol mixed fortunes', 'lost subscribers fourth quarter profits lower preceding three quarters', 'however company said aol underlying profit exceptional items rose back stronger internet advertising revenues', 'hopes increase subscribers offering online service free timewarner internet customers try sign aol existing customers high speed broadband', 'timewarner also restate results following probe us securities exchange commission sec close concluding', 'time warner fourth quarter profits slightly better analysts expectations', 'film division saw profits slump helped 

In [0]:
# vectorizing sentences
# we take the average of word vectors as sentence vector
# note that word vectors are of length 100
zero_vec = np.zeros((100,))
def get_sentence_vectors(text):
  ret = []
  for s in text:
    if len(s) >0 :
      words = s.split()
      avg_vec = np.mean([word_emb.get(w,zero_vec) for w in words],0)
      ret.append(avg_vec)
    else:
      ret.append(zero_vec)
  return np.array(ret)


sentence_vectors = deepcopy(clean_sentences)
for category,texts in clean_sentences.items():
  for i in range(len(texts)):
    sentence_vectors[category][i] = get_sentence_vectors(texts[i])


In [0]:
print(sentence_vectors["business"][0])

[[ 0.24647717  0.13236724  0.17578991 ...  0.07944505  0.6691668
  -0.13111262]
 [ 0.25836852  0.08868428  0.09233036 ... -0.20717452  0.73004615
   0.21344498]
 [ 0.13221574  0.14556488  0.2931974  ...  0.01991749  0.57184434
  -0.2566259 ]
 ...
 [ 0.01733747  0.11718404  0.11662699 ... -0.1930045   0.37169042
  -0.00848834]
 [ 0.21365242 -0.07733981  0.17945156 ... -0.02334311  0.6738694
   0.23502226]
 [ 0.34443122  0.182348    0.5066116  ... -0.11145437  0.5981372
   0.24595124]]


#Similarity Matrix#
* Implement cosine,unit overlap, and others

In [0]:
def get_similarity_matrix(sen_vec):
  num_sentences = sen_vec.shape[0]
  similarity_matrix = np.zeros((num_sentences,)*2)
  for i in range(num_sentences):
    for j in range(num_sentences):
      if i==j:
        # ignore self similarity
        continue;
      similarity_matrix[i][j] = cosine_similarity(sen_vec[i].reshape(1,100),
                                                  sen_vec[j].reshape(1,100))[0][0]
  return similarity_matrix
similarity_matrices = deepcopy(sentence_vectors)
for category,texts in sentence_vectors.items():
  for i in range(len(texts)):
    # here texts[i] is a list of sentence vectors
    similarity_matrices[category][i] = get_similarity_matrix(texts[i])

In [0]:
print(similarity_matrices["business"][0])

[[0.         0.89925611 0.87145078 0.89484477 0.82644361 0.79190433
  0.88884628 0.92014909 0.7916562  0.83178121 0.91676897 0.87958807
  0.90647203 0.79136986 0.88116074 0.81461054 0.7956745  0.81737548
  0.87152338 0.85676879]
 [0.89925611 0.         0.74273962 0.91161221 0.83683413 0.88447785
  0.81545109 0.94604707 0.89964169 0.78342092 0.87405753 0.81744307
  0.80953205 0.78997701 0.88832623 0.820122   0.76927036 0.81066215
  0.8732748  0.82911271]
 [0.87145078 0.74273962 0.         0.78734899 0.67249161 0.6240443
  0.82458907 0.79127139 0.62673223 0.68000931 0.82386166 0.72861052
  0.9165957  0.64636242 0.79851735 0.61898148 0.60613608 0.66212976
  0.70523077 0.74159408]
 [0.89484477 0.91161221 0.78734899 0.         0.73874545 0.83951336
  0.84482276 0.91905344 0.81016576 0.70736945 0.91018081 0.77999681
  0.84090441 0.69925678 0.91256934 0.76933712 0.69506717 0.73382521
  0.82659584 0.83360261]
 [0.82644361 0.83683413 0.67249161 0.73874545 0.         0.76745552
  0.70770067 0.84

#Applying Ranking Algorithm#


In [0]:
def get_ranked_sentences(sim_mat,sen):
  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  r_sen = sorted(((scores[i],i,s) for i,s in enumerate(sen)), reverse=True)
  r_sen = [(i,y) for (x,i,y) in r_sen]
  return r_sen


ranked_sentences = deepcopy(sentences)
for category,texts in sentences.items():
  for i in range(len(texts)):
    ranked_sentences[category][i] = get_ranked_sentences(similarity_matrices[category][i],sentences[category][i])



In [0]:
print("-------------:predicted:-------------")
sen = sorted(ranked_sentences["business"][1][:6])
for i,s in sen[:6]:
  print(s+'\n')
  # print in increasing order of i
  
print("\n-------------:actual:-------------")
print(summaries["business"][1])


-------------:predicted:-------------
Dollar gains on Greenspan speech

The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.

On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data.

"He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next."

In the meantime, the US Federal Reserve's decision on 2 February to boost interest rates by a quarter of a point - the sixth such move in as many months - has opened up a differential with European rates.

The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and gover

#Results#
* print 
* rouge and other measures to evaluate performance

In [0]:
listOfHypothesis = []
listOfReferences = []
# testing only for business
# extract top 6 sentences
max_top = 3
for i in range(len(ranked_sentences["business"])):
  sen = (sorted(ranked_sentences["business"][i][:max_top]))
  sen = [s for i,s in sen]
  sen = '\n'.join(sen)
  listOfHypothesis.append(sen)
  listOfReferences.append(summaries["business"][i])


In [0]:
print(listOfHypothesis[0])
print("---------------")
print(listOfReferences[0])

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.
The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.
However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.
---------------
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it no

In [0]:
!pip3 install rouge
import rouge
r = rouge.Rouge()
r.get_scores(listOfHypothesis, listOfReferences, avg=True)['rouge-1']['r']

0.3224343493191521

In [0]:
# # lsa
# from sklearn.feature_extraction.text import TfidfVectorizer

# def get_term_doc_matrix(X):
#   vectorizer = TfidfVectorizer(stop_words='english',
#   max_df = 0.5,smooth_idf=True)
#   doc_term = vectorizer.fit_transform(X)
#   n,m = doc_term.shape
#   term_doc = np.zeros((m,n))
#   for i in range(m):
#     # print(i,'/',m)
#     for j in range(n):
#       term_doc[i,j] = doc_term[(j,i)]
#   return term_doc

# td_hyp = get_term_doc_matrix(listOfHypothesis)
# td_ref = get_term_doc_matrix(listOfReferences)



In [0]:
# def get_lsa_vector(term_doc,r=1):
#   u,s,vh = np.linalg.svd(term_doc,full_matrices=False)
#   u_imp = u[:,:r]@np.diag(s[:r])
#   s = np.sqrt(np.sum(u_imp**2,1))
#   return s
 
# def cosine(u,v):
#   return np.sum(u*v)/(np.sum(u**2)*np.sum(v**2))

# max_terms = 5000
# u = get_lsa_vector(td_hyp[:max_terms,:])
# v = get_lsa_vector(td_ref[:max_terms,:])

# print(cosine(u,v))


In [0]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [0]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

In [0]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

In [0]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    # print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel.print_topics(num_topics=number_of_topics, num_words=words)[0][1]

In [0]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [0]:
def LSAError(hyp,ref):
  number_of_topics=1
  words =10
  clean_text = preprocess_data([hyp])
  key_words = create_gensim_lsa_model(clean_text,number_of_topics,words)
  hyp_words = [word.split('*') for word in key_words.split('+')]
  hyp_dict = {}
  for x in hyp_words:
    hyp_dict[x[1]]=x[0]

  clean_text = preprocess_data([ref])
  key_words = create_gensim_lsa_model(clean_text,number_of_topics,words)
  ref_words = [word.split('*') for word in key_words.split('+')]
  ref_dict = {}
  for x in ref_words:
    ref_dict[x[1]]=x[0]

  MSE = 0
  for key in hyp_dict:
    if key in ref_dict:
      MSE += (float(hyp_dict[key])-float(ref_dict[key]))**2
    else:
      MSE += (float(hyp_dict[key]))**2
  return MSE
  

The Keywords of the two summaries (predicted and given) differ by around : 21.7 %.
<br>
So loosely speaking they are 78.3% similar. 

In [0]:
number_of_topics=1
words=10
# document_list,titles=load_data("","articles.txt")
clean_text=preprocess_data([listOfHypothesis[1]])
key_words=create_gensim_lsa_model(clean_text,number_of_topics,words)
print(key_words)
# print([word.split('*') for word in key_words.split('+')])
# print(clean_text)

clean_text=preprocess_data([listOfReferences[1]])
key_words=create_gensim_lsa_model(clean_text,number_of_topics,words)
print(key_words)
# lst = [word.split('*') for word in key_words.split('+')]
# print(lst)
# dictionary = {}
# for x in lst:
#   dictionary[x[1]]=x[0]
# print(dictionary)
# print(LSAError(listOfHypothesis[1],listOfReferences[1]))

lsa_loss = 0
for i in range(len(listOfHypothesis)):
  lsa_loss += LSAError(listOfHypothesis[i],listOfReferences[i])
print(lsa_loss/len(listOfHypothesis))
# print(clean_text)

0.429*"us" + 0.214*"feder" + 0.214*"dollar" + 0.214*"month" + 0.214*"deficit" + 0.214*"rate" + 0.214*"reserv" + 0.107*"almost" + 0.107*"level" + 0.107*"buy"
0.354*"us" + 0.310*"dollar" + 0.310*"deficit" + 0.177*"recent" + 0.177*"month" + 0.133*"currenc" + 0.133*"feder" + 0.133*"chines" + 0.133*"greenspan" + 0.133*"reserv"
0.21394330196078434
